In [1]:
import cv2
import numpy as np
from keras.models import load_model


net = cv2.dnn.readNet("F:/CNN PROJECT/TEST FOLDER/TEST1/frozen_inference_graph.pb","F:/CNN PROJECT/TEST FOLDER/TEST1/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt")

behavior_model = load_model("F:/CNN PROJECT/TEST FOLDER/TEST1/behavenot50.h5")
emotion_model = load_model("F:/CNN PROJECT/TEST FOLDER/TEST1/emo84%.h5")   



behavior_labels = ['drinking','eating','sitting','sleeping','texting']  
emotion_labels = ['anger','hapiness','sad']        


cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    
    resized_frame = cv2.resize(frame, (80, 80))
    resized_frame = np.expand_dims(resized_frame, axis=0)

    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    
    num_persons = 0


    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  
            class_id = int(detections[0, 0, i, 1])
            if class_id == 1:  
                num_persons += 1

                
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (startX, startY, endX, endY) = box.astype("int")

                
                cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255,0), 2)

                
                roi = frame[startY:endY, startX:endX]

        
                resized_roi = cv2.resize(roi, (80, 80))
                resized_roi = np.expand_dims(resized_roi, axis=0)

                
                behavior_prediction = behavior_model.predict(resized_roi)[0]
                behavior_label = behavior_labels[np.argmax(behavior_prediction)]

            
                emotion_prediction = emotion_model.predict(resized_roi)[0]
                emotion_label = emotion_labels[np.argmax(emotion_prediction)]

        
                text = f'Behavior: {behavior_label}, Emotion: {emotion_label}'
                cv2.putText(frame, text, (startX, startY - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0, 0), 2)

    cv2.putText(frame, f'Number of Persons: {num_persons}', (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)

    cv2.imshow('Human Detection and Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 20ms/step
